In [1]:
#standard imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json, math, time
from tqdm.notebook import tqdm_notebook
import tmdbsimple as tmdb

In [2]:
with open('/Users/chas/.secret/tmdb_api.json', 'r') as f:
    login = json.load(f)
login.keys()

dict_keys(['api-key'])

In [3]:
tmdb.API_KEY = login['api-key']

In [4]:
#save API call data in data folder created in part 1
import os
FOLDER = "Data/"
os.makedirs(FOLDER, exist_ok=True)
os.listdir(FOLDER)

['title_basics.csv.gz',
 '.ipynb_checkpoints',
 'title_akas.csv.gz',
 'title_ratings.csv.gz']

In [5]:
#load dataframe from part 1
basics = pd.read_csv('Data/title_basics.csv.gz')
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El Tango del Viudo y Su Espejo Deformante,0,2020,NaN,70,Drama
2,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,NaN,122,Drama
3,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005,NaN,100,"Comedy,Horror,Sci-Fi"
4,tt0093119,movie,Grizzly II: Revenge,Grizzly II: The Predator,0,2020,NaN,74,"Horror,Music,Thriller"


In [6]:
#Define the years we are currently seeking
YEARS_TO_GET = [2000,2001]

In [7]:
def write_json(new_data, filename):
    with open(filename, 'r+') as file:
        #load existing data
        file_data = json.load(file)
        #choose extend or append
        if (type(new_data)==list) & (type(file_data)==list):
            file_data.extend(new_data)
        else:
            file_data.append(new_data)
        #set files current position at offset
        file.seek(0)
        #convert back to json
        json.dump(file_data, file)

In [8]:
def get_movie_with_rating(movie_id):
    #get the movie and release dates
    movie = tmdb.Movies(movie_id)
    ##contruct output dict
    movie_info = movie.info()
    releases = movie.releases()
    for c in releases['countries']:
        if c['iso_3166_1'] == 'US':
            movie_info['certification'] = c['certification']
    return movie_info

In [9]:
#set up outer loop
for YEAR in tqdm_notebook(YEARS_TO_GET, desc='YEARS',
                          position=0):
    #define the JSON file to store results for year
    JSON_FILE = f'{FOLDER}tmdb_api_results_{YEAR}.json'
    #check if file exists
    file_exists = os.path.isfile(JSON_FILE)

    #if it does not exist: create it
    if file_exists==False:
        with open(JSON_FILE,'w') as f:
            json.dump([{'imdb_id':0}], f)

    #save new year as the current df
    df = basics.loc[basics['startYear']==YEAR].copy()
    #save movie ids to list
    movie_ids = df['tconst'].copy()
    movie_ids

    #load existing data from json into dataframe
    previous_df = pd.read_json(JSON_FILE)

    #filter out any IDS in JSON_FILE
    movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]

    for movie_id in tqdm_notebook(movie_ids_to_get, desc=f'Movies from {YEAR}', 
                                 position = 1, leave=True):
        try:
            temp = get_movie_with_rating(movie_id)
            #append/extend results to existing file
            write_json(temp, JSON_FILE)
            time.sleep(0.02)

            #if it fails
        except Exception as e:
            continue
    final_year_df = pd.read_json(JSON_FILE)
    final_year_df.to_csv(f'{FOLDER}final_tmdb_data_{YEAR}.csv.gz',compression='gzip', index=False)

YEARS:   0%|          | 0/2 [00:00<?, ?it/s]

Movies from 2000:   0%|          | 0/1403 [00:00<?, ?it/s]

Movies from 2001:   0%|          | 0/1517 [00:00<?, ?it/s]